In [1]:
import os
import sys

STOCKA_BASE_DIR = "/home/zzy/projects/stocka"
sys.path.insert(0, STOCKA_BASE_DIR)

import pandas as pd

from datetime import date
from quant_framework.data.data_handler import DataHandler
from quant_framework.factor.alpha158 import Alpha158

In [3]:
data_handler = DataHandler(
    db_path=os.path.join(STOCKA_BASE_DIR, 'data/stock.db'),
)

In [8]:
for factor_id, factor_name, factor_category, factor_desc in Alpha158.DEFINITIONS:
    data_handler.register_factor(factor_id, factor_name, factor_category, factor_desc)

In [10]:
end_date = date(2026, 2, 5)
start_date = data_handler.get_previous_trading_date(end_date, 60)
start_date

datetime.date(2025, 11, 11)

In [ ]:
from tqdm import tqdm

factor_df = None
code_list = data_handler.get_all_codes()
# code_list = ['000001', '000002', '600000']
pbar = tqdm(code_list)
alpha158 = Alpha158()
for code in pbar:
    pbar.set_postfix({"code": code})
    df = data_handler.get_range_data([code], start_date, end_date)
    new_factor_row = alpha158.calculate(df).iloc[[-1]]
    if factor_df is None:
        factor_df = new_factor_row
    else:
        factor_df = pd.concat([factor_df, new_factor_row], ignore_index=True)

 18%|█▊        | 541/3063 [00:58<04:35,  9.17it/s, code=002050]

In [7]:
factor_df

,code,date,open,high,low,close,volume,amount,outstanding_share,turnover,...,VSUMN5,VSUMN10,VSUMN20,VSUMN30,VSUMN60,VSUMD5,VSUMD10,VSUMD20,VSUMD30,VSUMD60
0,000001,2005-01-04,179.01,179.01,175.48,177.11,1760832.0,1.146560e+07,1.409362e+09,0.001249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000001,2005-01-05,177.11,177.92,172.49,175.48,3222144.0,2.071856e+07,1.409362e+09,0.002286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000001,2005-01-06,176.56,179.01,175.20,177.11,2666413.0,1.733384e+07,1.409362e+09,0.001892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000001,2005-01-07,178.73,179.28,175.48,176.83,1886151.0,1.230285e+07,1.409362e+09,0.001338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000001,2005-01-10,176.83,179.01,173.03,179.01,2632055.0,1.711150e+07,1.409362e+09,0.001868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14920,600000,2026-01-30,168.40,170.72,166.24,166.73,110436836.0,1.119322e+09,3.330584e+10,0.003316,...,0.547818,0.521377,0.481629,0.472868,0.515710,-0.095637,-0.042754,0.036741,0.054265,-0.031421
14921,600000,2026-02-02,167.23,170.22,166.57,167.07,105771232.0,1.072786e+09,3.330584e+10,0.003176,...,0.599887,0.451173,0.511362,0.479892,0.506831,-0.199773,0.097653,-0.022724,0.040217,-0.013662
14922,600000,2026-02-03,167.07,167.23,164.58,166.40,77669459.0,7.755098e+08,3.330584e+10,0.002332,...,0.546438,0.494273,0.528988,0.494736,0.504798,-0.092875,0.011455,-0.057976,0.010528,-0.009596
14923,600000,2026-02-04,166.07,168.56,165.74,168.23,67248961.0,6.789721e+08,3.330584e+10,0.002019,...,0.731993,0.541517,0.502261,0.490662,0.507218,-0.463986,-0.083034,-0.004521,0.018676,-0.014436


In [27]:
factor_names = [d[0] for d in Alpha158.DEFINITIONS]
data_handler.save_factors(factor_df[factor_names], "2026-02-05")

In [28]:
data_handler.close()